In [1]:
%load_ext autoreload

In [ ]:
%autoreload 2
%matplotlib inline
import os
from collections import namedtuple
from scipy.stats import gamma, bernoulli
import mmap
import pandas as pd
from itertools import chain
import numpy as np
from scipy.stats import scoreatpercentile
import re
# from laslib import Las
# from googtool import Google
import holoviews as hv
import easier as ezr
hv.extension('bokeh')


import pylab as pl


import warnings
warnings.filterwarnings("ignore")

In [ ]:
%opts Curve Scatter [width=700, height=400]
%opts Overlay [legend_position='top', show_grid=True] 

In [ ]:
x = np.linspace(0, 4 * np.pi, 1000)
xa = 180 * x / np.pi
yc = np.sin(x)
yb = .25 * np.sin(x - 3 * np.pi / 4)
ym = yc + yb

ax = ezr.figure(style='seaborn-talk', grid=True)
ax.plot(xa, yc, label='Cable Signal')
ax.plot(xa, yb, label='Background Signal')
ax.plot(xa, ym, label='Measured Signal')
ax.legend(loc='best')
ax.set_xlim(0, 720)
ax.set_ylim(-1.5, 1.8)
pl.xlabel('Bearing')
pl.ylabel('Signal Strength')

file_name = 'sigs_vs_bearing.pdf'
pl.figure(ax.figure.number)
pl.savefig(file_name)




In [1]:
!pwd

/Users/rob/rob/repos/braket/acculocate_workflow_final/figure_creation_files


In [1]:

from scipy.stats import gamma, bernoulli
import numpy as np


def bplus(s, m):
    return np.abs(m + s)


def bminus(s, m):
    return np.abs(m - s)


def sample_amplitude(mu: float, sigma: float, N: int):
    """
    Draw samples from a model of either the signal or the
    background amplitude distribution.
    Args:
        mu: the mean value of the amplitude model
        sigma: the standard deviation of the amplitude model
        N: the number of random points to generate
    """
    # Compute the gamma parameters that will have the desired
    # mean and standard deviation
    a = (mu / sigma) ** 2
    scale = sigma ** 2 / mu

    # Generate the random variables.
    gam = gamma(a, scale=scale)
    x = gam.rvs(size=N)
    return x
    
def sample_angle(measured, sig_mu, sig_std, back_mu, back_std, N, max_iter=100):
    x_list = []
    n_samples = N
    n_iter = 0
    while n_samples > 0 and n_iter < max_iter:
        signal = sample_amplitude(sig_mu, sig_std, n_samples)
        background = sample_amplitude(back_mu, back_std, n_samples)
        
        
        numerator = measured ** 2 + signal ** 2 - background ** 2
        denominator = 2 * measured * signal
        
        ratio = numerator / denominator
        ratio = ratio[ratio < 1]
        theta_abs = np.arccos(ratio)
        nn = len(theta_abs)
        signs = 2 * (bernoulli(.5).rvs(nn) - .5)
        theta = theta_abs * signs
        theta = theta[~np.isnan(theta)]
        x_list.extend(theta.tolist())
        n_samples = int(2 * (N - len(x_list)))
        n_iter += 1
    if n_iter >= max_iter:
        raise ValueError(
            'MaxIter encounterd.  '
            'You probably set a model that is incompatible with measurement')
    return 180 * np.array(x_list[:N]) / np.pi


# Specify the parameters
measured = 2000
sig_mu=1900
back_mu=900 
sig_std = 200
back_std = 200
N = 100000

# Generate the samples
samples = sample_angle(measured, sig_mu, sig_std, back_mu, back_std, N, max_iter=100)


In [2]:
samples

array([-28.43017239,  23.5568365 ,  30.07050984, ..., -15.87591666,
       -25.58437811, -31.36360009])

In [ ]:
ax = ezr.figure(style='seaborn-whitegrid', grid=True)
ax.hist(x, bins=50, edgecolor='black', linewidth=1.2)
pl.xlabel('Bearing Correction Angle (deg)')
pl.ylabel('Counts')
ax.set_yticklabels([])

file_name = 'correction_hist.pdf'
pl.figure(ax.figure.number)
pl.savefig(file_name)
_;

In [ ]:
%opts Curve [logx=False, logy=False, show_grid=True, tools=['hover']]
thetadeg = np.linspace(8, 89, 3000)
theta = thetadeg * np.pi / 180
sbr = 1. / np.sin(theta)
hv.Curve(
    (thetadeg, sbr),
    hv.Dimension('Max Tolerable Bearing Error (deg)', range=(0, 25)),
    hv.Dimension('Min Allowable Signal-to-Background', range=(.1, 60))
)
# hv.Curve(
#     (sbr, thetadeg),
#     hv.Dimension('Min Allowable Signal-to-Background'),
#     hv.Dimension('Max Tolerable Bearing Error (deg)'),
# )